In [6]:
import pandas as pd

In [7]:
with open('customer_data.csv', 'r') as file:
    print(file.encoding)

UTF-8


In [ ]:
import 

In [2]:
import codecs
import os
with codecs.open('customer_data.csv', 'r') as file:
    BLOCKSIZE = 512
    if not os.path.exists('customer_data(utf-8).csv'):
        with codecs.open('customer_data(utf-8).csv', "w", "utf-8") as targetFile:
            while True:
                contents = file.read(BLOCKSIZE)
                if not contents:
                    break
                targetFile.write(contents)

In [9]:
file = pd.read_csv('customer_data(utf-8).csv', index_col =1, nrows=130000)

In [10]:
file

,會員卡號,資料日期,資料時間,餐別帶,縣市別,店舖代號,主商圈,品號-品名稱,群號-群名稱,單品名稱,銷售數量,銷售單價,交易金額,性別,年齡
交易id,,,,,,,,,,,,,,,
01637920171201184556000112534201,NaN,2017-12-01,18:45:56,晚餐時間帶,台中市,11148,住宅型,72-家庭雜貨,722-家雜用品,（新）銷售用購物袋１８號袋,1,1.0,1.0,NaN,NaN
01564720171201184641000113754802,NaN,2017-12-01,18:46:41,晚餐時間帶,台中市,3860,住宅型,00-傳統代收,11-信用卡,花旗信用卡,1,0.0,0.0,NaN,NaN
01614720171201163042000114405402,NaN,2017-12-01,16:30:42,下午茶時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,飲冰室茶集綠奶茶,1,25.0,25.0,NaN,NaN
01614720171201072745000114390002,NaN,2017-12-01,07:27:45,早餐時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,義美奶茶,1,35.0,35.0,NaN,NaN
01614720171201065809000118172701,NaN,2017-12-01,06:58:09,早餐時間帶,台中市,10697,住宅型,28-其他乳品,286-發酵乳,養樂多,1,8.0,8.0,NaN,NaN
01614720171201065009000114387302,NaN,2017-12-01,06:50:09,早餐時間帶,台中市,10697,住宅型,27-鮮乳,271-大包裝全脂鮮乳,瑞穗全脂鮮乳９３０ｍｌ,1,87.0,87.0,NaN,NaN
01384020171201130558000119295301,NaN,2017-12-01,13:05:58,午餐時間帶,彰化縣,13840,文教型,12-調理麵,122-熱食麵２配,沙茶牛肉炒麵,1,69.0,69.0,NaN,NaN
01384020171201130034000119295201,NaN,2017-12-01,13:00:34,午餐時間帶,彰化縣,13840,文教型,68-保健衛生,682-醫療用品,醫療平面口罩１０入,1,79.0,79.0,NaN,NaN
01384020171201114808000119291701,NaN,2017-12-01,11:48:08,午餐時間帶,彰化縣,13840,文教型,61-啤酒,612-進口啤酒,麒麟霸啤酒３３０ＭＬ,3,32.0,88.0,NaN,NaN


In [5]:
def aggregate(df):
    dic = {}
    for index, row in df.iterrows():
        if index not in dic:
            dic[index] = []
        dic[index].append(str(row['單品名稱']))
    return dic

In [33]:
purchase_list = aggregate(file)

# Apriori

In [19]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

ModuleNotFoundError: No module named 'mlxtend'

In [28]:
te = TransactionEncoder()
te_ary = te.fit(purchase_list.values()).transform(purchase_list.values())
df = pd.DataFrame(te_ary, columns=te.columns_)

In [29]:
df

,nan,一口爆漿餡餅（５入）,一度贊爌肉麵（碗）,一度贊爌肉麵（袋）,一度贊麻辣牛肉麵（碗）,一日水果１００％蘋果汁,一日蔬果１００％紫色蔬果汁,一日蔬果１００％蔬果汁,一條根精油貼布（薑黃）,一番榨生啤酒－５００,...,ｅＴａｇ儲值,ｅＴａｇ繳費,ｅｚ店到店手續費,ｅｚ簡訊寄件手續費,ｋｉｔｋａｔ提拉米蘇巧克力,ｋｉｔｋａｔ酥脆花生奶油巧克力,ｋｉｔｋａｔ黑巧克力,ｏｒｉｏｎ生啤酒,ｐｉｎｋｙ水蜜桃薄荷糖,ｓｕｐｅｒ冰炫薄荷口香糖
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
result = apriori(df, min_support=0.001, use_colnames=True)
result.sort_values(by=['support'], ascending=False)

,support,itemsets
319,0.027607,(蝦皮取件Ｃ)
292,0.023704,(茶葉蛋（銷售用）)
406,0.019679,(（新）銷售用購物袋１８號袋)
287,0.015694,(自由時報)
7,0.012970,(七星１０毫克硬盒香煙)
389,0.012807,(麥香奶茶ＴＰ３００)
208,0.011222,(熱拿鐵大杯)
95,0.010449,(台灣自來水)
25,0.010205,(伯朗咖啡)
277,0.009839,(義美奶茶)


In [17]:
result.shape[0]

446

In [12]:
result["items_sets_len"] = result["itemsets"].apply(lambda x: len(x))
multiple_item_sets = result[result['items_sets_len'] > 1]
single_item_sets = result[(result['items_sets_len'] == 1) & (result['support'] > 0.001)]

In [13]:
multiple_item_sets

,support,itemsets,items_sets_len
428,0.001301,"(中信外１５, 代收手續費１５)",2
429,0.001423,"(台灣自來水, 中華電信)",2
430,0.001016,"(寶物交易代, 代收手續費２５)",2
431,0.004594,"(健保費代收, 代收手續費４)",2
432,0.001098,"(台灣自來水, 代收手續費４)",2
433,0.004188,"(國民年金代, 代收手續費４)",2
434,0.001260,"(通行繳費, 代收手續費５)",2
435,0.001138,"(台中裁罰單, 代收手續費６)",2
436,0.001667,"(奶香綠茶３３０ＭＬ, 代收折價卷)",2
437,0.002480,"(促銷券０６, 伊藤園蘋果紅茶)",2


In [14]:
single_item_sets

,support,itemsets,items_sets_len
0,0.006953,(七星天藍７毫克硬盒),1
1,0.001952,(七星天藍７毫克軟包),1
2,0.001260,(七星酷藍７毫克香煙),1
3,0.001586,(七星酷黑１０毫克香煙),1
4,0.001057,(七星雙味晶球５毫克),1
5,0.001626,(七星雙味晶球８毫克涼菸),1
6,0.001830,(七星風藍４毫克硬盒),1
7,0.012970,(七星１０毫克硬盒香煙),1
8,0.003537,(七星１０毫克軟包),1
9,0.001342,(中信外１５),1


# Association Rules

In [54]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(result, metric="confidence", min_threshold=0.6)

In [56]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(中信外１５),(代收手續費１５),0.001342,0.004025,0.001301,0.969697,240.906030,0.001296,32.867168
1,(代收手續費２５),(寶物交易代),0.001138,0.001016,0.001016,0.892857,878.392857,0.001015,9.323846
2,(寶物交易代),(代收手續費２５),0.001016,0.001138,0.001016,1.000000,878.392857,0.001015,inf
3,(健保費代收),(代收手續費４),0.004676,0.008620,0.004594,0.982609,113.996514,0.004554,57.004371
4,(國民年金代),(代收手續費４),0.004188,0.008620,0.004188,1.000000,116.014151,0.004152,inf
5,(代收手續費５),(通行繳費),0.001830,0.001260,0.001260,0.688889,546.555556,0.001258,3.210234
6,(通行繳費),(代收手續費５),0.001260,0.001830,0.001260,1.000000,546.555556,0.001258,inf
7,(台中裁罰單),(代收手續費６),0.001138,0.001952,0.001138,1.000000,512.395833,0.001136,inf
8,(奶香綠茶３３０ＭＬ),(代收折價卷),0.002236,0.004025,0.001667,0.745455,185.196511,0.001658,3.912758
9,(伊藤園蘋果紅茶),(促銷券０６),0.002805,0.003822,0.002480,0.884058,231.312828,0.002469,8.592036


# Network Analysis

In [20]:
import networkx as nx

In [21]:
Graph = nx.Graph()

In [37]:
def convert(purchase_list):
    def find_edges_in_list(itemsets):
        from itertools import combinations
        result = []
        return combinations(itemsets, 2)
    result = {}
    for key, itemsets in purchase_list.items():
        if len(itemsets) > 4:
            for edge in find_edges_in_list(itemsets):
                if edge in result:
                    result[edge] += 1
                else:
                    result[edge] = 1
    return result

In [38]:
edges_dict = convert(purchase_list)

In [47]:
edges_dict

{('（新）銷售用購物袋１８號袋', '瑞穗低脂鮮乳９３０ｍｌ'): 1,
 ('（新）銷售用購物袋１８號袋', '鮮奶油布蕾'): 2,
 ('（新）銷售用購物袋１８號袋', '統一大布丁（雞蛋口味）'): 6,
 ('（新）銷售用購物袋１８號袋', '鮮奶茶'): 5,
 ('瑞穗低脂鮮乳９３０ｍｌ', '鮮奶油布蕾'): 1,
 ('瑞穗低脂鮮乳９３０ｍｌ', '統一大布丁（雞蛋口味）'): 1,
 ('瑞穗低脂鮮乳９３０ｍｌ', '鮮奶茶'): 1,
 ('鮮奶油布蕾', '統一大布丁（雞蛋口味）'): 1,
 ('鮮奶油布蕾', '鮮奶茶'): 1,
 ('統一大布丁（雞蛋口味）', '鮮奶茶'): 1,
 ('代收折價卷', '波的多洋芋片蚵仔煎口味'): 1,
 ('代收折價卷', '奶香綠茶３３０ＭＬ'): 9,
 ('代收折價卷', '經典木瓜（截切）'): 1,
 ('代收折價卷', '熱拿鐵中杯'): 1,
 ('波的多洋芋片蚵仔煎口味', '奶香綠茶３３０ＭＬ'): 1,
 ('波的多洋芋片蚵仔煎口味', '經典木瓜（截切）'): 1,
 ('波的多洋芋片蚵仔煎口味', '熱拿鐵中杯'): 1,
 ('奶香綠茶３３０ＭＬ', '經典木瓜（截切）'): 1,
 ('奶香綠茶３３０ＭＬ', '熱拿鐵中杯'): 1,
 ('經典木瓜（截切）', '熱拿鐵中杯'): 1,
 ('甘樂鮮果實軟糖－白葡萄口味', 'ＵＨＡ酷露露果實糖檸檬軟糖'): 1,
 ('甘樂鮮果實軟糖－白葡萄口味', 'ＵＨＡ噗啾麝香葡萄軟糖'): 1,
 ('甘樂鮮果實軟糖－白葡萄口味', '麥香錫蘭奶茶'): 1,
 ('甘樂鮮果實軟糖－白葡萄口味', '芒果冰茶'): 1,
 ('ＵＨＡ酷露露果實糖檸檬軟糖', 'ＵＨＡ噗啾麝香葡萄軟糖'): 1,
 ('ＵＨＡ酷露露果實糖檸檬軟糖', '麥香錫蘭奶茶'): 1,
 ('ＵＨＡ酷露露果實糖檸檬軟糖', '芒果冰茶'): 1,
 ('ＵＨＡ噗啾麝香葡萄軟糖', '麥香錫蘭奶茶'): 1,
 ('ＵＨＡ噗啾麝香葡萄軟糖', '芒果冰茶'): 1,
 ('麥香錫蘭奶茶', '芒果冰茶'): 1,
 ('關東煮本舖拉麵', '白玉蘿蔔'): 22,
 ('關東煮本舖拉麵', '海鮮魚卵棒'): 23,
 ('關東煮本舖拉麵', '黃

In [53]:
def construct_graph_from_edges(edges_list):
    graph = Graph = nx.Graph()
    for edge, weight in edges_list.items():
        if weight >= 5:
            graph.add_edge(*edge, weight=weight)
            graph.add_nodes_from(edge)
    return graph

In [54]:
graph = construct_graph_from_edges(edges_dict)

In [55]:
print('Edge Count: {}\nNode Count: {}'.format(len(graph.edges), len(graph.nodes)))

Edge Count: 316
Node Count: 150


In [56]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

plot = Plot(plot_width=1440, plot_height=1440,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Graph Interaction Demonstration"

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("club", "@club")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(graph, nx.spring_layout, scale=1.5, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

In [57]:
from bokeh.io import show, output_file